In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import STL
import matplotlib.font_manager as fm
import seaborn as sns
import math
import re
from sklearn.preprocessing import LabelEncoder

In [2]:
# 파일 읽기
df21 = pd.read_csv("C:/Users/etbae/Downloads/mainC/train_subway21.csv")

# 역 이름 인코딩
le = LabelEncoder()
df21['train_subway21.station_name_encoded'] = le.fit_transform(df21['train_subway21.station_name'])

# 방향: 바이너리 인코딩
df21['train_subway21.direction_bin'] = df21['train_subway21.direction'].map({'내선': 0, '상선': 0, '외선': 1, '하선': 1})

# 날짜 분할
df21['train_subway21.datetime'] = pd.to_datetime(df21['train_subway21.tm'], format='%Y%m%d%H')

# 수치형 열만 선택
numeric_cols = df21.select_dtypes(include=[np.number]).columns
# 각 열에 대해 이상치 처리: -99 이하인 값은 NaN으로
for col in numeric_cols:
    df21[col] = df21[col].map(lambda x: np.nan if isinstance(x, (int, float)) and x <= -99 else x)
# 추가 처리: wd 값이 -9.9이면 NaN
df21['train_subway21.wd'] = df21['train_subway21.wd'].map(lambda x: np.nan if x == -9.9 else x)


##### * 보간 방식 선택: 사용하지 않는 방식을 주석처리 *

In [3]:
# 1. 정렬 --------------------------------------------------------------
df21 = df21.sort_values("train_subway21.datetime")

# 2. 대상 컬럼 ---------------------------------------------------------
cols_to_clean = ["train_subway21.ta", "train_subway21.ta_chi", "train_subway21.ws"]

# 3. datetime을 인덱스로 설정 ------------------------------------------
df21 = df21.set_index("train_subway21.datetime")

# 4. 보간 방식 선택 (주석처리로 둘 중 하나 선택) ------------------------
# --- 방법 1: 시간 기준 선형 보간 (index 간격 활용) ---
df21[cols_to_clean] = df21[cols_to_clean].interpolate(method="time", limit_direction="both")

# --- 방법 2: 3차 스플라인 보간 ---
# df21[cols_to_clean] = df21[cols_to_clean].interpolate(method="spline", order=3, limit_direction="both")

# 5. 인덱스 복구 -------------------------------------------------------
df21 = df21.reset_index()

# 6. 확인 --------------------------------------------------------------
print(df21[cols_to_clean].isna().sum())  # 남은 NaN 개수


train_subway21.ta        0
train_subway21.ta_chi    0
train_subway21.ws        0
dtype: int64


In [4]:
# 1. 정렬 --------------------------------------------------------------
df21 = df21.sort_values("train_subway21.datetime")

# 2. 대상 컬럼 ---------------------------------------------------------
cols_to_clean = ["train_subway21.hm"]

# 3. datetime을 인덱스로 설정 ------------------------------------------
df21 = df21.set_index("train_subway21.datetime")

# 4. 보간 방식 선택 (주석처리로 둘 중 하나 선택) ------------------------
# --- 방법 1: 시간 기준 선형 보간 ---
df21[cols_to_clean] = df21[cols_to_clean].interpolate(method="time", limit_direction="both")

# --- 방법 2: 3차 스플라인 보간 ---
# df21[cols_to_clean] = df21[cols_to_clean].interpolate(method="spline", order=3, limit_direction="both")

# 5. 인덱스 복구 -------------------------------------------------------
df21 = df21.reset_index()

# 6. 확인 --------------------------------------------------------------
print(df21[cols_to_clean].isna().sum())  # 남은 NaN 개수


train_subway21.hm    0
dtype: int64


In [5]:
# 1. 대상 컬럼 ---------------------------------------------------------
col = 'train_subway21.rn_hr1'

# 2. 정렬 -------------------------------------------------------------
df21 = df21.sort_values("train_subway21.datetime")

# 3. 인덱스 설정 ------------------------------------------------------
df21 = df21.set_index('train_subway21.datetime')

# 4. 결측 여부 및 마스크 정의 ------------------------------------------
isna = df21[col].isna()
left0 = df21[col].shift(1).fillna(0) == 0
right0 = df21[col].shift(-1).fillna(0) == 0
zero_mask = isna & (left0 | right0)
interpolate_mask = isna & (~zero_mask)

# 5. 결측값 처리 -------------------------------------------------------
# (1) 양쪽이 0인 경우 → 0으로 대체
df21.loc[zero_mask, col] = 0

# (2) 나머지 → 보간
# 보간 방식 선택 (주석처리로 둘 중 하나 선택) ------------------------
# --- 방법 1: 시간기반 보간 ---
df21.loc[interpolate_mask, col] = df21[col].interpolate(method='time', limit_direction='both')[interpolate_mask]

# --- 방법 2: 스플라인 보간 ---
# df21.loc[interpolate_mask, col] = df21[col].interpolate(method='spline', order=3, limit_direction='both')[interpolate_mask]

# 6. 인덱스 복구 ------------------------------------------------------
df21 = df21.reset_index()

# 7. 결과 확인 --------------------------------------------------------
missing_count = df21[col].isna().sum()
print(f"[{col}] 결측치 남은 개수:", missing_count)


[train_subway21.rn_hr1] 결측치 남은 개수: 0


In [6]:
import pandas as pd
import numpy as np

# 1. 보간 대상 컬럼 및 시간정보 생성 ------------------------------
col = 'train_subway21.si'

df21['month'] = df21['train_subway21.datetime'].dt.month
df21['hour'] = df21['train_subway21.datetime'].dt.hour

# 2. 월-시 기준 NaN 비율 계산 후 밤 시간대 추정 ------------------
threshold = 0.9  # 밤으로 간주할 NaN 비율 기준

si_null_ratio = (
    df21.groupby(['month', 'hour'])[col]
    .apply(lambda x: x.isna().mean())
    .reset_index(name='null_ratio')
)

# 월별 밤 시간대 목록 추출
night_hours_by_month = (
    si_null_ratio[si_null_ratio['null_ratio'] >= threshold]
    .groupby('month')['hour']
    .agg(list)
    .to_dict()
)

# 3. 낮/밤 플래그 생성 -------------------------------------------
def is_daytime(row):
    return row['hour'] not in night_hours_by_month.get(row['month'], [])

df21['is_daytime'] = df21.apply(is_daytime, axis=1)

# 4. 밤 시간대 NaN → 0으로 대체 ----------------------------------
df21.loc[~df21['is_daytime'] & df21[col].isna(), col] = 0

# 5. 낮 시간대 NaN → 전체 보간 ------------------------------------
# 정렬 및 인덱스 설정
df21 = df21.sort_values("train_subway21.datetime")
df21 = df21.set_index('train_subway21.datetime')

# 낮 시간대 마스크
day_mask = df21['is_daytime'] & df21[col].isna()

# 보간 방식 선택 (주석처리로 둘 중 하나 선택) ------------------------
# --- 방법 1: 시간 기반 보간 ---
df21.loc[day_mask, col] = df21[col].interpolate(method='time', limit_direction='both')[day_mask]

# --- 방법 2: 3차 스플라인 보간 ---
# df21.loc[day_mask, col] = df21[col].interpolate(method='spline', order=3, limit_direction='both')[day_mask]

# 인덱스 복구
df21 = df21.reset_index()

# 6. 남은 결측치 확인 --------------------------------------------
missing_count = df21[col].isna().sum()
print(f"[{col}] 남은 결측치 개수:", missing_count)


[train_subway21.si] 남은 결측치 개수: 0


In [7]:
# 1. 날짜 컬럼 생성 --------------------------------------------------
# (기존에 있으면 생략 가능)
df21['date'] = df21['train_subway21.datetime'].dt.date

# 2. 대상 컬럼 지정 -------------------------------------------------
rn_day_col = 'train_subway21.rn_day'
rn_hr1_col = 'train_subway21.rn_hr1'

# 3. 일별 rn_hr1 합계 계산 ------------------------------------------
# groupby는 필수 (일 단위 합산이 목적이므로)
sum_rn_hr1 = (
    df21.groupby(['train_subway21.station_name_encoded', 'train_subway21.station_number', 'date'])[rn_hr1_col]
    .sum()
    .reset_index()
    .rename(columns={rn_hr1_col: 'sum_rn_hr1'})
)

# 4. 원본 데이터에 일별 합계 merge ----------------------------------
df21 = df21.merge(
    sum_rn_hr1,
    on=['train_subway21.station_name_encoded', 'train_subway21.station_number', 'date'],
    how='left'
)

# 5. rn_day 결측치 대체 --------------------------------------------
# rn_day가 결측이면 sum_rn_hr1 값으로 대체
mask = df21[rn_day_col].isna()
df21.loc[mask, rn_day_col] = df21.loc[mask, 'sum_rn_hr1']

# 6. 결과 확인 ------------------------------------------------------
missing_count = df21[rn_day_col].isna().sum()
print(f"[{rn_day_col}] 남은 결측치 개수:", missing_count)


[train_subway21.rn_day] 남은 결측치 개수: 0


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

df21['day'] = df21['train_subway21.datetime'].dt.day

# --- 사용자 지정 입력 변수와 타겟 -------------------------------
target_col = "train_subway21.congestion"
feature_cols = ['train_subway21.line', 'train_subway21.ta', 'train_subway21.ws', 'train_subway21.rn_hr1', 'train_subway21.hm', 'train_subway21.si',
                'train_subway21.ta_chi', 'train_subway21.station_name_encoded', 'train_subway21.direction_bin', 'month', 'hour', 'day']  # 원하는 변수만 넣기

# --- 데이터 전처리 ----------------------------------------------
df_model = df21[feature_cols + [target_col]].dropna()
X = df_model[feature_cols]
y = df_model[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# --- 모델 정의 ---------------------------------------------------
models = {
    "LightGBM": lgb.LGBMRegressor(),
    "XGBoost": xgb.XGBRegressor(),
    "HistGradientBoosting": HistGradientBoostingRegressor(),
    #"RandomForest": RandomForestRegressor()
}

# --- 결과 저장 ---------------------------------------------------
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)

    results.append({
        "Model": name,
        "R²": round(r2, 4),
        "RMSE": round(rmse, 4)
    })

# --- 결과 출력 ----------------------------------------------------
results_df = pd.DataFrame(results)
print("모델 성능 비교:")
print(results_df.to_string(index=False))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026488 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1550
[LightGBM] [Info] Number of data points in the train set: 4361714, number of used features: 12
[LightGBM] [Info] Start training from score 18.289881
모델 성능 비교:
               Model     R²    RMSE
            LightGBM 0.4167 13.8685
             XGBoost 0.4850 13.0316
HistGradientBoosting 0.4126 13.9179


In [11]:
# csv 파일로 저장
df21.to_csv("C:/Users/etbae/Downloads/임시/train_subway21_nogroupby_time.csv", index=False, encoding='utf-8-sig')

In [12]:
# 요일, 시간 파생
df21['weekday'] = df21['train_subway21.datetime'].dt.weekday  # 0=월, ..., 6=일
df21['hour'] = df21['train_subway21.datetime'].dt.hour

# 평일 여부 (월~금 = 1, 토/일 = 0)
df21['train_subway21.is_weekday'] = df21['weekday'].apply(lambda x: 1 if x < 5 else 0)

# 혼잡 시간대 여부 (출근 07–09시, 퇴근 17–19시)
df21['train_subway21.rush_hour'] = df21['hour'].apply(lambda h: 1 if h in [7, 8, 9, 17, 18, 19] else 0)


In [13]:
# csv 파일로 저장
df21.to_csv("C:/Users/etbae/Downloads/임시/train_subway21_nogroupby_time_added.csv", index=False, encoding='utf-8-sig')